In [6]:
# Run setup code
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tltk
tltk.pos_load()
%matplotlib inline

In [7]:
# Path to the preprocessed data
best_processed_path = 'w_review_train.csv'

In [8]:
# Create a character map
CHARS = [
  '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
  ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
  '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
  'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
  'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
  'n', 'o', 'other', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
  'z', '}', '~', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช',
  'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท',
  'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ',
  'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า',
  'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', 'เ', 'แ', 'โ', 'ใ', 'ไ',
  'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๐', '๑', '๒', '๓',
  '๔', '๕', '๖', '๗', '๘', '๙', '‘', '’', '\ufeff'
]
CHARS_MAP = {v: k for k, v in enumerate(CHARS)}
print(CHARS_MAP)
print(len(CHARS_MAP))

{'5': 22, 'ู': 149, 'x': 89, 'X': 57, '๖': 171, 'R': 51, 'a': 65, '+': 12, 'q': 82, '&': 7, 'น': 118, ']': 62, "'": 8, 'f': 70, 'u': 86, '/': 16, '\ufeff': 177, '\n': 0, '4': 21, 'd': 68, 'H': 41, '"': 3, 'ฮ': 138, 'ฅ': 98, 'c': 67, 'ฤ': 129, '๙': 174, 'ข': 95, 'M': 46, 'h': 72, 'ห': 135, 'แ': 152, '์': 163, '้': 160, 'ม': 126, 'ฺ': 150, '1': 18, '๊': 161, 'ั': 141, 'ป': 120, 'B': 35, 'o': 79, ':': 27, 'i': 73, '๘': 173, '‘': 175, 'S': 52, 'ํ': 164, 'Q': 50, '%': 6, 'G': 40, '๔': 169, 'l': 76, 'ผ': 121, 'ุ': 148, 'ร': 128, 'F': 39, 'Y': 58, 'โ': 153, 'ฑ': 110, 'W': 56, 'ๆ': 157, 'ะ': 140, 'ฎ': 107, 'ี': 145, 'ฏ': 108, '?': 32, '๒': 167, 'ด': 113, 'ถ': 115, 'C': 36, 'ิ': 144, 'g': 71, 'P': 49, '_': 64, 'D': 37, 'า': 142, '[': 60, 'k': 75, '(': 9, 'ึ': 146, 'ฟ': 124, 'O': 48, 'ว': 131, 'ภ': 125, 'ื': 147, 'other': 80, 's': 84, '*': 11, 'พ': 123, 'ใ': 154, 'j': 74, 'I': 42, '#': 4, 'p': 81, 'n': 78, 'ฐ': 109, ';': 28, '7': 24, 'y': 90, '๗': 172, 'อ': 137, 'T': 53, '!': 2, 'N': 47, ')': 10

In [9]:
from keras.models import Model
from keras.layers import Embedding,Dense, Conv1D, Flatten, TimeDistributed, Dropout
from keras.layers import Input, GRU, Bidirectional
from keras.optimizers import Adam
# def get_my_best_model2():
#     input1 = Input(shape=(21,))
#     x = Embedding(178,8)(input1)
#     x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
#     x = TimeDistributed(Dense(5))(x)ฅ
#     x = Flatten()(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     x = Dropout(0.05)(x)
#     x = Dense(100, activation='relu')(x)
#     out = Dense(1, activation='sigmoid')(x)
#     model = Model(inputs=input1, outputs=out)
#     model.compile(optimizer=Adam(),
#                  loss='binary_crossentropy',
#                  metrics=['acc'])          
#     return model

def get_my_best_model():
    input1 = Input(shape=(21,))
    x = Embedding(178,8)(input1)
    x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
    x = TimeDistributed(Dense(5))(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                 loss='binary_crossentropy',
                 metrics=['acc'])          
    return model
    

Using TensorFlow backend.


In [10]:
weight_path_my_best_model2='/data/model_best.h5'
my_best_model2 = get_my_best_model()
my_best_model2.load_weights(weight_path_my_best_model2)
#my_best_model2.make_predict_function()
my_best_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 21, 8)             1424      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 21, 100)           4100      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 5)             505       
_________________________________________________________________
flatten_1 (Flatten)          (None, 105)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10600     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
__________

In [11]:
df = pd.read_csv(best_processed_path, sep=';', header=None)

In [12]:
df

,0,1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...,4
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...,5
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...,5


In [13]:
def create_n_gram_df(df, n_pad):
  """
  Given an input dataframe, create a feature dataframe of shifted characters
  Input:
  df: timeseries of size (N)
  n_pad: the number of context. For a given character at position [idx],
    character at position [idx-n_pad/2 : idx+n_pad/2] will be used 
    as features for that character.
  
  Output:
  dataframe of size (N * n_pad) which each row contains the character, 
    n_pad_2 characters to the left, and n_pad_2 characters to the right
    of that character.
  """
  n_pad_2 = int((n_pad - 1)/2)
  for i in range(n_pad_2):
      df['char-{}'.format(i+1)] = df[0].shift(i + 1)
      df['char{}'.format(i+1)] = df[0].shift(-i - 1)
  return df[n_pad_2: -n_pad_2]

In [14]:
word_test = [x for x in df[0][1]]
print(''.join(word_test))

อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา ตัวร้านตั้งอยู่ที่ถนนพุทธมณฑลสาย 3 ไปตาม ถ.ปิ่นเกล้า-นครชัยศรี เมื่อถึงพุทธมณฑลสาย 3 ก็เลี้ยวเข้าไปประมาณ 500 เมตร ร้านอยู่ทางซ้ายมือค่ะ มีคนบอกมาว่าความพิเศษของร้านนี้คือกุ๊กเก่าและเป็นกุ๊กรุ่นสุดท้ายจาก "ฮก ลก ซิ่ว” ภัตตาคารจีนชื่อดังย่านราชประสงค์ ที่เลิกกิจการไปแล้ว ต้องคนที่อายุเลข 5 ขึ้นไปจึงจะเคยกิน ฮก ลก ซิ่ว  จานเด็ดที่มีขายที่นี่แห่งเดียวในเมืองไทยคือ ปลาเต๋าเต้ย 2 ฤดู เป็นสูตรจากมาเลเซีย นอกนั้นก็มี ผัดผักน้ำมันหอย ไก่เบตง เคาหยก ปูทะเลซุปน้ำใสหม้อไฟ เต้าหู้แคระยัดไส้หม้อดิน และ ลูกชิ้นแคระ 
อาหารที่เราแนะนำคือไก่เบตง (คล้ายๆไก่แช่เหล้า) เสริฟพร้อมกับหอมเจียว และน้ำจิ้มน้ำพริกเผาสูตรเด็ดของทางร้าน 
เมนูข้าวผัดหนำเลียบก็อร่อยค่ะ ชอบมากๆ


In [15]:
word_test_df2 = pd.DataFrame(word_test)
#print(word_test_df2)

n_pad = 21
n_pad_2 = int((n_pad - 1)/2)
pad = [{0: ' '}]
df_pad = pd.DataFrame(pad * n_pad_2)
#print(df_pad)

word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
#print(word_test_df2[0][:21])
word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))

In [16]:
df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
             ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

# convert pandas dataframe to numpy array to feed to the model
x_char = df_with_context[char_row].as_matrix()

In [17]:
y_pred = my_best_model2.predict(x_char)

prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
y_pred = np.apply_along_axis(prob_to_class,1,y_pred)

In [18]:
print(len(y_pred))
print(len(word_test))
word_token = []
word_tmp = ''
for i in range(len(y_pred)):
    if(y_pred[i] == 1):
        word_token.append(word_tmp)
        word_tmp = ''
    word_tmp += word_test[i]
else:
    word_token.append(word_tmp)
word_token = word_token[1:]
print(word_token)

701
701
['อาหาร', 'ที่', 'นี่', 'เป็น', 'อาหาร', 'จีน', 'แคะ', 'ที่', 'หากิน', 'ยาก', 'ใน', 'บ้าน', 'เรา', ' ', 'ตัว', 'ร้าน', 'ตั้ง', 'อยู่', 'ที่', 'ถนนพุทธมณฑล', 'สาย', ' ', '3', ' ', 'ไป', 'ตาม', ' ', 'ถ.', 'ปิ่นเกล้า', '-', 'นครชัยศรี', ' ', 'เมื่อ', 'ถึง', 'พุทธมณฑล', 'สาย', ' ', '3', ' ', 'ก็', 'เลี้ยว', 'เข้า', 'ไป', 'ประมาณ', ' ', '500', ' ', 'เมตร', ' ', 'ร้าน', 'อยู่', 'ทาง', 'ซ้าย', 'มือ', 'ค่ะ', ' ', 'มี', 'คน', 'บอก', 'มา', 'ว่า', 'ความ', 'พิเศษ', 'ของ', 'ร้าน', 'นี้', 'คือ', 'กุ๊ก', 'เก่า', 'และ', 'เป็น', 'กุ๊ก', 'รุ่น', 'สุดท้าย', 'จาก', ' ', '"', 'ฮก', ' ', 'ลก', ' ', 'ซิ่ว', '”', ' ', 'ภัตตา', 'คาร', 'จีน', 'ชื่อ', 'ดัง', 'ย่าน', 'ราชประสงค์', ' ', 'ที่', 'เลิก', 'กิจการ', 'ไป', 'แล้ว', ' ', 'ต้อง', 'คน', 'ที่', 'อายุ', 'เลข', ' ', '5', ' ', 'ขึ้น', 'ไป', 'จึง', 'จะ', 'เคย', 'กิน', ' ', 'ฮก', ' ', 'ลก', ' ', 'ซิ่ว', ' ', ' ', 'จาน', 'เด็ด', 'ที่', 'มี', 'ขาย', 'ที่', 'นี่', 'แห่ง', 'เดียว', 'ใน', 'เมืองไทย', 'คือ', ' ', 'ปลาเต๋าเต้ย', ' ', '2', ' ', 'ฤดู', ' ', 'เป็น'

In [19]:
def cutKumKrub(string):
    word_test = [x for x in string]
    word_test_df2 = pd.DataFrame(word_test)

    n_pad = 21
    n_pad_2 = int((n_pad - 1)/2)
    pad = [{0: ' '}]
    df_pad = pd.DataFrame(pad * n_pad_2)

    word_test_df2 = pd.concat((df_pad, word_test_df2, df_pad))
    word_test_df2[0] = word_test_df2[0].map(lambda x: CHARS_MAP.get(x, 80))
    
    df_with_context = create_n_gram_df(word_test_df2, n_pad=n_pad)

    char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
                 ['char-' + str(i + 1) for i in range(n_pad_2)] + [0]

    # convert pandas dataframe to numpy array to feed to the model
    x_char = df_with_context[char_row].as_matrix()
    
    y_pred = my_best_model2.predict(x_char)

    prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
    y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    
    word_token = []
    word_tmp = ''
    for i in range(len(y_pred)):
        if(y_pred[i] == 1):
            word_token.append(word_tmp)
            word_tmp = ''
        word_tmp += word_test[i]
    else:
        word_token.append(word_tmp)
    word_token = word_token[1:]
    return word_token

In [20]:
def mergeLabel(review, rating):
    noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
    cleaned = list(filter(lambda x: len(x.replace(" \n", "")) > 0 and len(x.replace(")", "")) > 0, noSpace)) 
    for i in range(len(cleaned)):
        cleaned[i] = cleaned[i].replace("\n", "")
        cleaned[i] = cleaned[i].replace("(", "")
        cleaned[i] = cleaned[i].replace(")", "")
    finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, cleaned))
    label = [int(rating)] * len(cleaned)
    training_data = list(zip(cleaned, label))
    return training_data

In [21]:
mergeLabel(df[0][1], df[1][1])

[('อาหาร', 4),
 ('ที่', 4),
 ('นี่', 4),
 ('เป็น', 4),
 ('อาหาร', 4),
 ('จีน', 4),
 ('แคะ', 4),
 ('ที่', 4),
 ('หากิน', 4),
 ('ยาก', 4),
 ('ใน', 4),
 ('บ้าน', 4),
 ('เรา', 4),
 ('ตัว', 4),
 ('ร้าน', 4),
 ('ตั้ง', 4),
 ('อยู่', 4),
 ('ที่', 4),
 ('ถนนพุทธมณฑล', 4),
 ('สาย', 4),
 ('3', 4),
 ('ไป', 4),
 ('ตาม', 4),
 ('ถ.', 4),
 ('ปิ่นเกล้า', 4),
 ('-', 4),
 ('นครชัยศรี', 4),
 ('เมื่อ', 4),
 ('ถึง', 4),
 ('พุทธมณฑล', 4),
 ('สาย', 4),
 ('3', 4),
 ('ก็', 4),
 ('เลี้ยว', 4),
 ('เข้า', 4),
 ('ไป', 4),
 ('ประมาณ', 4),
 ('500', 4),
 ('เมตร', 4),
 ('ร้าน', 4),
 ('อยู่', 4),
 ('ทาง', 4),
 ('ซ้าย', 4),
 ('มือ', 4),
 ('ค่ะ', 4),
 ('มี', 4),
 ('คน', 4),
 ('บอก', 4),
 ('มา', 4),
 ('ว่า', 4),
 ('ความ', 4),
 ('พิเศษ', 4),
 ('ของ', 4),
 ('ร้าน', 4),
 ('นี้', 4),
 ('คือ', 4),
 ('กุ๊ก', 4),
 ('เก่า', 4),
 ('และ', 4),
 ('เป็น', 4),
 ('กุ๊ก', 4),
 ('รุ่น', 4),
 ('สุดท้าย', 4),
 ('จาก', 4),
 ('"', 4),
 ('ฮก', 4),
 ('ลก', 4),
 ('ซิ่ว', 4),
 ('”', 4),
 ('ภัตตา', 4),
 ('คาร', 4),
 ('จีน', 4),
 ('ชื่อ', 4),
 ('ดั

In [22]:
demo = df[:][:1000]
demo

,0,1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5
5,ร้านข้าวต้มกระดูกหมู ปากซอยพัฒนาการ 57 เป็นอีก...,4
6,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
7,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
8,เมื่อวันก่อนไปเดินเล่น แบบชิวๆๆ ที่สยามสแควร์แ...,5
9,วันก่อนไปเดินสยาม หลังจากห่างหายไป ประมาณ 6 เด...,5


In [23]:
train = []
for i in range(len(demo)):
    train.append((demo[0][i], demo[1][i]))

In [24]:
# train

In [25]:
def remove_htmlTags(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext
    
def isValidWord(text):
    dic = {text[0]:1}
    for c in text[1:]:
        if c not in dic:
            return True
        else:
            dic[c] += 1
    if dic[text[0]] > 1:
        return False
    else:
        return True
        
def tokenize(review):
    
#     tags = tltk.nlp.pos_tag(review.replace("\n", ""))
#     noSpace = list(filter(lambda x: len(x.replace(" ", "")) > 0, cutKumKrub(review)))
#     cleaned = list(filter(lambda x: len(x.replace(" \n", "")) > 0 and len(x.replace(")", "")) > 0, noSpace)) 
#     for i in range(len(cleaned)):
#         cleaned[i] = cleaned[i].replace("\n", "")
#         cleaned[i] = cleaned[i].replace("(", "")
#         cleaned[i] = cleaned[i].replace(")", "")
#     finalWord = list(filter(lambda x: len(x.replace(" ", "")) > 0, cleaned))

    # Filter parts of speech of text
    tags = tltk.nlp.pos_tag(remove_htmlTags(review.replace("\n", "")))
    finalWord = []
    filteredTags = {'NOUN', 'VERB', 'ADJ','ADV', 'AUX', 'PART'}
    for sentence in tags:
        for word, tag in sentence:
            if tag in filteredTags:
                if len(word) > 1 and isValidWord(word):
                    finalWord.append(word)
    return finalWord

# tokenize(train[0][0])
# train[0][0]

In [26]:
# from itertools import chain
# vocabulary = set(chain(*[tokenize(i[0].lower()) for i in train]))

In [27]:
# vocabulary
tokenize(train[i][0])

['ไม่',
 'รู้จัก',
 'ร้าน',
 'หรอก',
 'ครับ',
 'ไม่ค่อย',
 'ได้',
 'มา',
 'แถว',
 'พี่',
 'สาว',
 'แฟน',
 'ชวน',
 'มา',
 'กิน',
 'เปิด',
 'ดู',
 'เว็บ',
 'หา',
 'แผนที่',
 'ร้าน',
 'เห็น',
 'รูป',
 'ร้าน',
 'สวย',
 'มาก',
 'ครับ',
 'มี',
 'โต๊ะ',
 'ริม',
 'น้ำ',
 'ไม่',
 'มาก',
 'ต้อง',
 'รีบ',
 'จอง',
 'ห้อง',
 'แอร์',
 'มี',
 'เหมือนกัน',
 'ด้าน',
 'นอก',
 'บรรยากาศ',
 'ดี',
 'ครับ',
 'อาหาร',
 'ราคา',
 'ค่อนข้าง',
 'สูง',
 'ปริมาณ',
 'เยอะ',
 'รสชาติ',
 'ดี',
 'ครับ',
 'กุ้ง',
 'หอบ',
 'ปู',
 'ปลา',
 'สด',
 'ดี',
 'มี',
 'คุณภาพ',
 '<Fail>พนักงานบริิการดีครับ</Fail>',
 'มี',
 'อารมณ์ขัน',
 'ด้วย',
 'อาหาร',
 'จะ',
 'นาน',
 'ไป',
 'บ้าง',
 'พนักงาน',
 'ไม่',
 'ชัก',
 'สีห',
 'ร้า',
 'เวลา',
 'โดน',
 'ทวงถาม',
 'บ่อยๆ']

In [ ]:
from IPython.display import clear_output
corpus = []
for i in range(len(train)):
    print(str(i+1) + "/" + str(len(train)))
    corpus.append(tokenize(train[i][0]))

corpus
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

1/1000
2/1000
3/1000
4/1000
5/1000
6/1000

In [ ]:

import pickle


pickle_out = open("corpus.pickle","wb")
pickle.dump(corpus, pickle_out)
pickle_out.close()


In [ ]:
# pickle_in = open("corpus.pickle","rb")
# corpus = pickle.load(pickle_in)
    
# corpus

In [ ]:
corpus
newcor = corpus[:700]
test = corpus[700:]
# tltk.nlp.pos_tag(train[0][0])

In [ ]:
tfidf = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, binary = True)
tfidf_matrix = tfidf.fit_transform(newcor)

In [ ]:
tfidf_matrix

In [ ]:

idf = tfidf.idf_
print (tfidf.get_feature_names(), idf)

In [ ]:
# corpus = [
#     tokenize(df[0][0]),
#     tokenize(df[0][1]),
#     tokenize(df[0][2])
# ]

In [ ]:
x_train = tfidf_matrix.toarray()
y_train = []

for i in range(700):
    y_train.append(train[i][1])
y_train = np.array(y_train)

In [ ]:
x_train.shape

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import f1_score

classifier = SVC(kernel = 'rbf')
linearClassifier = SVC(kernel = 'linear')
# classifer = GaussianNB()
classifier.fit(x_train, y_train)
linearClassifier.fit(x_train, y_train)

In [ ]:
tfidf = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x, binary = True)
tfidf_matrix = tfidf.transform(test)
x_test = tfidf_matrix.toarray()
y_test = []

for i in range(700,1000):
    y_test.append(train[i][1])
y_test = np.array(y_test)

predictions = classifier.predict(x_test)

In [ ]:
# x_train.mean()
# predictions
f1_score(y_test, predictions, average='macro')

In [ ]:
predictions = linearClassifier.predict(x_train)
predictions
f1_score(y_test, predictions, average='macro')